In [35]:
project_name = "agi_agi"
path_tps_file = f"/data/ATM/data_1/sfm/agi_projects/{project_name}/debug/tie_points.csv"

In [36]:
import pandas as pd
df = pd.read_csv(path_tps_file)

print(df.columns)

Index(['tp_id', 'x_left', 'x_right', 'y_left', 'y_right', 'size_left',
       'size_right', 'keypoint_left', 'keypoint_right', 'image_id_left',
       'image_id_right', 'track_id'],
      dtype='object')


In [37]:
# round the x and y coordinates to 1 decimal place
df['x_left'] = df['x_left'].round(1)
df['y_left'] = df['y_left'].round(1)
df['x_right'] = df['x_right'].round(1)
df['y_right'] = df['y_right'].round(1)

# Create separate dataframes for left and right
left_columns = ['tp_id', 'x_left', 'y_left', 'size_left', 'keypoint_left', 'image_id_left', 'track_id']
right_columns = ['tp_id', 'x_right', 'y_right', 'size_right', 'keypoint_right', 'image_id_right', 'track_id']

# Rename left and right columns
df_left = df[left_columns].rename(columns={
    'x_left': 'x', 'y_left': 'y', 'size_left': 'size', 'keypoint_left': 'keypoint', 'image_id_left': 'image_id'
})
df_right = df[right_columns].rename(columns={
    'x_right': 'x', 'y_right': 'y', 'size_right': 'size', 'keypoint_right': 'keypoint', 'image_id_right': 'image_id'
})


# Concatenate the left and right dataframes
df_combined = pd.concat([df_left, df_right])
# Concatenate left and right data
df_combined = pd.concat([df_left, df_right])

# Create a dictionary where the keys are image_id and the values are individual DataFrames
dfs_per_image_id = {image_id: df_combined[df_combined['image_id'] == image_id] for image_id in df_combined['image_id'].unique()}


In [40]:
import numpy as np
from scipy.spatial import distance_matrix

tolerance = 1

for key in dfs_per_image_id.keys():
    df = dfs_per_image_id[key]
    coordinates = df[['x', 'y']].values

    # Compute the distance matrix (pairwise Euclidean distances)
    dist_matrix = distance_matrix(coordinates, coordinates)
    
    # Find pairs of points where the distance is less than or equal to the tolerance (excluding self-pairs)
    common_points = np.where((dist_matrix <= tolerance) & (dist_matrix > 0))
    
        # Create a DataFrame to show pairs of common points
    common_df = pd.DataFrame({
        'tp_id_1': df.iloc[common_points[0]]['tp_id'].values,
        'tp_id_2': df.iloc[common_points[1]]['tp_id'].values,
        'x_1': df.iloc[common_points[0]]['x'].values,
        'y_1': df.iloc[common_points[0]]['y'].values,
        'x_2': df.iloc[common_points[1]]['x'].values,
        'y_2': df.iloc[common_points[1]]['y'].values,
        'image_id': df.iloc[common_points[0]]['image_id'].values,
        'distance': dist_matrix[common_points]
    })
    print("KEY", key)
    print(common_df)
    

KEY 0.0
Empty DataFrame
Columns: [tp_id_1, tp_id_2, x_1, y_1, x_2, y_2, image_id, distance]
Index: []
KEY 1.0
Empty DataFrame
Columns: [tp_id_1, tp_id_2, x_1, y_1, x_2, y_2, image_id, distance]
Index: []
KEY 2.0
   tp_id_1  tp_id_2     x_1     y_1     x_2     y_2  image_id  distance
0   1899.0    831.0  5290.2  2882.5  5290.3  2883.0       2.0  0.509902
1    831.0   1899.0  5290.3  2883.0  5290.2  2882.5       2.0  0.509902
KEY 3.0
   tp_id_1  tp_id_2     x_1     y_1     x_2     y_2  image_id  distance
0   1727.0   1971.0  1645.1  4117.8  1644.8  4118.0       3.0  0.360555
1   1971.0   1727.0  1644.8  4118.0  1645.1  4117.8       3.0  0.360555
